In [41]:
!gdown --id '1qCHfycy91EyUFzilBvxu8hYVYp0l6jCh' --output /tmp/MidTerm_Dataset.zip

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qCHfycy91EyUFzilBvxu8hYVYp0l6jCh
To: /tmp/MidTerm_Dataset.zip
100% 50.4M/50.4M [00:00<00:00, 337MB/s]


In [42]:
import os
import zipfile
local_zip = '/tmp/MidTerm_Dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [43]:
%cd ..
%cd /tmp/MidTerm_Dataset

/tmp
/tmp/MidTerm_Dataset


In [44]:
import numpy as np
import random
import os
import keras
from PIL import Image #PIL 提供處理image的模組
from keras.models import Sequential #建立最簡單的線性模型 (Sequentia1），就是一層層往下執行，沒有分叉 (If)
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D #CNN 的捲積層和池化層
from keras.utils import np_utils #用來後續將 Label 標籤轉為 one-hot-encoding
from matplotlib import pyplot as plt #用來繪製優化過程曲線的套件


In [45]:
def data_x_y_preprocess(datapath):
  img_row, img_col = 28, 28 # 定義圖片大小
  datapath = datapath # 訓練資料路徑
  data_x = np.zeros ( (28,28) ).reshape (1, 28, 28) # 儲存圖片
  pictureCount = 0 # 紀錄圖片張敟
  data_y = [] #紀錄label
  workpiece_class = 4 # 工件種類有10種
  # 讀取image資料夾内所有檔案
  for root, dirs, files in os.walk(datapath):
    for f in files:
      label = int (root.split(os.sep) [2]) #取得1abel
      data_y.append(label)
      fullpath = os.path.join(root, f) #取得檔案路徑
      img = Image.open(fullpath) #開啟img
      img = (np.array(img)/255).reshape(1, 28,28) #讀取資料時順便做正規化典reshape
      data_x = np.vstack( (data_x, img) )
      pictureCount += 1
  data_x = np.delete(data_x, [0], 0) #删除一開始宣告的np.zeros
  #調整資料格式（圖片張數，img_row, img_col, 色彩通道=1（灰階)）
  data_x = data_x.reshape(pictureCount, img_row, img_col, 1)
  data_y = np_utils.to_categorical(data_y, workpiece_class) #將1abe1轉為one-hot encoding
  return data_x, data_y

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
#取得train_X和train_Y
data_train_X, data_train_Y = data_x_y_preprocess ("train_image") #, data y preprocess ("image")
#取得test_X和test_Y
data_test_X, data_test_Y = data_x_y_preprocess ("test_image")

In [48]:
from keras.layers.core.dense import activations
#建立簡單的線性執行的模型
model = Sequential()
#建立卷積層，filter=32
model.add (Conv2D (32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
#建立池化層，池化大小=2x2，取最大值
model.add (MaxPooling2D (pool_size=(2, 2)))
#建立卷積層，filter=64
model.add (Conv2D (64, (3, 3), activation='relu'))
#建立池化層，池化大小=2×2，取最大值
model.add (MaxPooling2D (pool_size=(2, 2)))


In [49]:
# Dropout層隨機断開輸入神經元，用於防止過度擬合，断開比例：0.1
model.add( Dropout(0.1) )
# Flatten層把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add( Flatten() )
# Dropout層隨機断開比例：0.1
model.add( Dropout(0.1) )
# 全連接層： 128 /output
model.add( Dense(128,activation='relu') )
# Dropout層隨機斷開比例：0.25
model.add( Dropout (0.25) )
# 使用 softmax activation function，將結果分類 (units=4 表示分4類）
model.add( Dense (units=4, activation='softmax') )

In [50]:
model.summary() # 查看模型架構  

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 5, 5, 64)          0         
                                                                 
 flatten_2 (Flatten)         (None, 1600)             

In [51]:
# 編譯：選擇損失函數、優化方法及成效衡量方式
model.compile (loss="categorical _crossentropy", optimizer="adam", metrics=[' accuracy'])

train_history = model.fit (data_train_X, data_train_Y, batch_size=32, epochs=100, verbose=1, validation_split=0.1)

ValueError: ignored